Here is the system that "broke" LU factorization for us.

In [ ]:
A = [2 0 4 3 ; -4 5 -7 -10 ; 1 15 2 -4.5 ; -2 0 2 -13];
b = [ 4, 9, 29, 40 ];

When we use the `lu` function (from `LinearAlgebra`) with three outputs, we get the elements of the PLU factorization.

In [ ]:
using LinearAlgebra
L,U,p = lu(A);
L

In [ ]:
U

In [ ]:
p

As you see above, the `p` return is a vector permutation of `1:n`, rather than the permutation matrix `P`. We can recover the latter as follows:

In [ ]:
I = diagm(0=>ones(4))
P = I[p,:]

However, this is rarely necessary in practice (and the vector requires a lot less storage). We can the linear system, for example, using only `p`.

In [ ]:
include("../FNC.jl")
x = FNC.backsub( U, FNC.forwardsub(L,b[p,:]) )

If you call `lu` with just one output, it is a "factorization object". You can access the individual parts of it using a dot syntax. 

In [ ]:
fact = lu(A)
fact.L

The factorization object can be used efficiently to solve linear systems by the backslash. 

In [ ]:
x = fact\b

The idea here is that if you have to solve many different linear systems for the same matrix, you can perform the computationally expensive factorization just once, and repeat only the much faster triangular solves for the different right-hand sides. 